# TSO Master Class Notebook, part 1 --- Target Observability

The objective of this first part of the "solution" TSO Master Class Notebook is to use the different tools available to perform target observability calculations for TSO observations. Part of the solutions are currently in ``python``; aside from a ``python`` installation, you will need the following dependencies:

- Numpy (https://numpy.org/), Scipy (https://www.scipy.org/), Matplotlib (https://matplotlib.org/).
- Astropy (https://www.astropy.org/).
- Astroplan (https://astroplan.readthedocs.io/en/latest/installation.html#id2).
- <a id='https://jwst-docs.stsci.edu/other-tools/target-visibility-tools/jwst-general-target-visibility-tool-help'>JWST General Target visibility tool</a>.

Optionally, you might want to install seaborn (https://seaborn.pydata.org/) for some pretty plots. Once you've installed these dependencies, you are good to go! 

Let's thus jump to the actual solutions. In this particular case, we will be trying to schedule a (or several) transit(s) of the exoplanet K2-18 b. Let us first check for possible transits in JWST's Cycle 1, and see if they are actually observable by JWST. 

**1. Looking for transits of K2-18 b**

To look for transits of the exoplanet, we will first use the `astroplan` library (installable through `pip install astroplan`) to search for transits (of course, Master Class atendees can use any tool they want!). We take the ephemerides of the system from <a id='https://arxiv.org/abs/1909.04642'>Bennekee et al. (2019)</a>, which are $t_0 = 2457527.91121$ and $P=32.940038$ days. The duration of the transit is $0.14$ days. Let's find then which transits are observable between the start (September 30, 2021) and end (September 30, 2022) of Cycle 1. For this, first create an object for our target with these ephemerides:

In [14]:
from astropy.time import Time
import astropy.units as u
from astroplan import EclipsingSystem

# Define ephemerides:
primary_eclipse_time = Time(2457527.91121, format='jd')
orbital_period = 32.940038 * u.day
eclipse_duration = 0.14 * u.day

# Create object for our target:
k218 = EclipsingSystem(primary_eclipse_time=primary_eclipse_time,
                       orbital_period=orbital_period, 
                       duration=eclipse_duration,
                       name='K2-18 b')

With our object created, let us now find when the next 11 eclipses (that's 329 days) occur for K2-18b since the start of Cycle 1:

In [15]:
# Define start of the observing window (start of Cycle 1 observations):
observing_time = Time('2021-09-30 00:00')
# Print dates for the next transits:
k218.next_primary_eclipse_time(observing_time, n_eclipses=11)

<Time object: scale='utc' format='iso' value=['2021-10-16 19:31:24.536' '2021-11-18 18:05:03.819'
 '2021-12-21 16:38:43.102' '2022-01-23 15:12:22.386'
 '2022-02-25 13:46:01.669' '2022-03-30 12:19:40.952'
 '2022-05-02 10:53:20.235' '2022-06-04 09:26:59.518'
 '2022-07-07 08:00:38.802' '2022-08-09 06:34:18.085'
 '2022-09-11 05:07:57.368']>

Perfect! We have the times of transit of the target. We now need to know if the target is actually observable by JWST during those dates.

**2. Can JWST observe all of those transits?**

Not every target is observable all the time by JWST. Only targets in the JWST Continuous Viewing Zone (CVZ) are observable all the time. To check the observability windows for all the JWST instruments we will use the <a id='https://jwst-docs.stsci.edu/other-tools/target-visibility-tools/jwst-general-target-visibility-tool-help'>JWST General Target visibility tool</a>. To use the tool, you can simply give the RA and DEC of the object along with start and end dates you wish to check, and you will receive a plot to check the observability of your target with JWST. Our target's coordinates are `172.560455` degrees in RA and `7.588391` degrees in DEC; running the JWST General Target Visibility Tool:

`jwst_gtvt 172.560455 7.588391 --name "K2-18" --start_date 2021-09-30 --end_date 2022-09-30`

Gives the following plot:

![Results from the JWST General Target Visibility tool](img/k218_jwst_gtvt.png)

So it seems we _cannot_ observe all the transits we want for this target. In fact, inspecting the figure and crossmatching it with our transit times obtained in the first part of this notebook, *we can see that only the transits on 2021-12-21 and 2022-01-23 are schedulable with V3 Position Angles (PA's) between about 285 and 298, and only the transits on 2022-05-02 and 2022-06-04 are schedulable with V3 PA's between about 110 and 125*. 

**3. The importance of Position Angles for transit spectroscopy**

Before ending this first part of the TSO Level 2 activity in terms of schedulability of transits, it is important to discuss about the importance of PAs for exoplanetary spectroscopic observations, which might also be important for other scientific cases as well. Most transit, eclipse and phase-curve exoplanet observations will be made with slitless modes, which implies that light from other nearby sources in the aperture might enter the instrument and contaminate our observations. This is important to try to avoid if possible because this will dilute the transit signatures in a wavelength-dependant way, which implies corrections will most likely have to be made to the data in 
a post-processing stage.

The ExoCTK team has provided a <a id='https://exoctk.stsci.edu/contam_visibility'>contamination & visibility calculator for this purpose</a>, which allow users to check the contamination level of the spectra to be obtained, and how to mitigate it by carefully selecting the PA of the observations. Currently (November 2019), this tool only works for the NIRISS SOSS mode, but it is anyways useful for other instruments as a guide at to what level of contamination to expect on our observations. Let us try it for our target. Aside from a visibility plot similar to the one shown above, this tool provides a contamination map that looks as follows:

![ExoCTK's contamination tool results](img/cont.png)

The outer panels in this plot show us what percentage of the wavelength chanel is being contaminated by nearby sources for a given PA, whereas the central part shows us how the contamination varies with wavelength for that same PA (with the top plot indicating important features that might appear in exoplanet spectra). In our case, PA's close to a 100 will produce some contamination, so we might want to avoid those. However, from this plot and the visibility plot, the conclusion is that most of the PAs in which our target is observable will give rise from small to no contamination. Our observations are "safe" in this sense!

